In [ ]:
import os
import logging
import together
from langchain.llms.base import LLM
from langchain import PromptTemplate, LLMChain
from dotenv import load_dotenv
load_dotenv()
logging.basicConfig(level=logging.INFO)

class TogetherLLM(LLM):
    model: str = "togethercomputer/llama-2-7b-chat"
    together_api_key: str = os.environ["TOGETHER_API_KEY"]
    temperature: float = 0.7
    max_tokens: int = 512

    def __init__(self, model=None, max_tokens=None, temperature=None):
        if model:
            self.model = model
        if max_tokens:
            self.max_tokens = max_tokens
        if temperature:
            self.temperature = temperature

    @property
    def llm_type(self) -> str:
        return "together"

    def __call__(self, prompt: str, **kwargs) -> str:
        try:
            logging.info("Calling Together endpoint.")
            return self.make_api_call(prompt)
        except Exception as e:
            logging.error(f"Error in TogetherLLM call: {e}", exc_info=True)
            raise

    def make_api_call(self, prompt: str) -> str:
        together.api_key = self.together_api_key
        output = together.Complete.create(
            prompt,
            model=self.model,
            max_tokens=self.max_tokens,
            temperature=self.temperature,
        )
        logging.info("API call successful.")
        return output['output']['choices'][0]['text']

# Now let's use the class
llm = TogetherLLM(
    model="togethercomputer/llama-2-7b-chat",
    max_tokens=256,
    temperature=0.8
)

prompt_template = "You are a friendly AI. Answer the following question: {question}"
prompt = PromptTemplate(
    input_variables=["question"], template=prompt_template
)
chat = LLMChain(llm=llm, prompt=prompt)


Integrating TogetherAI with LangChain 🦙

📌 `langchain.llms.base.LLM` is an abstract base class. The purpose of this class is to expose a simpler interface for working with LLMs, rather than expect the user to implement the full _generate method.

------------

### the significance of the this block of code

```py
  @property
    def _llm_type(self) -> str:
        """Return type of LLM."""
        return "together"
```


📌 This method, `_llm_type`, serves as a getter for a property of the `TogetherLLM` object. 

📌 The `@property` decorator transforms the `_llm_type` method to behave like an attribute of the `TogetherLLM` class, rather than a method that needs to be explicitly called. This means you can access the type of the LLM by referring to `instance._llm_type` rather than `instance._llm_type()`.

📌 The significance of defining this property in the class is primarily for internal use within the `TogetherLLM` or its parent classes. It could be used for type checking, logging, conditional processing based on the LLM type, or other purposes where identifying the type of the language model is necessary.

---------

### 📌 The reason for using `__call__` method above

The job of this method is to Check Cache and run the LLM on the given prompt and input.

📌 When you define the `__call__` method in a class, it enables you to use instances of that class like this: `instance(parameters)`, where `instance` is an object of the class.

📌 So with `__call__` the `TogetherLLM` class instances becomes callable objects. 

📌 That means, instead of having to explicitly call a method like `instance.call(prompt)`, you can simply use `instance(prompt)`. This makes the code more concise and can improve readability, especially for users who are familiar with functional programming paradigms.

📌 Another advantage is that it allows the `TogetherLLM` class to integrate more seamlessly with Python features and libraries that expect callable objects. For instance, if you're using a higher-order function that takes a function as an argument, you could directly pass an instance of `TogetherLLM` instead of having to wrap it in another function or lambda.